In [ ]:
# # Place the triples folder under the directory!

# !git clone https://github.com/ForeverHaibara/Triple-SOS.git
# !cp Triple-SOS/triples triples

!pip install sympy>=1.10 --user
!pip install scipy>=1.6 --user
!pip install numpy --user
!pip install clarabel --user  # if not successful, install cvxopt instead
# please RESTART the kernel after installation

# Triples

**GitHub** https://github.com/ForeverHaibara/Triple-SOS

<!-- Symbolic computation is based on SymPy. -->

The package helps prove algebraic inequalities via exact sum-of-squares.

In [4]:
from triples.utils import *
# from triples.core import sum_of_squares
from triples.core import *
from IPython.display import display
import sympy as sp
from sympy import sqrt, Rational
import numpy as np
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z = sp.symbols('a:z') # define SymPy symbols

## Basic Usage

Use `sum_of_squares(expr, ineq_constraints=[], eq_constraints=[])` to prove `expr >= 0`.

`Ineq_constraints` and `eq_constraints` are lists of symbolic expressions that are nonnegative and zero, respectively.

If it fails, it returns None. If it succeeds, it returns a Solution object.

The solution expression can be accessed by `.solution` property.

<!-- `sum_of_squares(expr: Expr, ineq_constraints: Union[List[Expr], Dict[Expr, Expr]], eq_constraints: Union[List[Expr], Dict[Expr, Expr]])` -->

#### Example: Nesbitt's Inequality

Goal: Prove that $\frac{a}{b+c}+\frac{b}{c+a}+\frac{c}{a+b} - \frac{3}{2}\geq 0$ given $a\geq 0,b\geq 0,c\geq 0$.

In [5]:
# Proving a/(b+c) + b/(c+a) + c/(a+b) >= 3/2 given a>=0, b>=0, c>=0
sol = sum_of_squares(CyclicSum(a/(b+c), (a,b,c)) - Rational(3,2), [a,b,c])
print(sol)
sol

Solution(problem = Σa/(b + c) - 3/2, solution = ((Σa*(b - c)**2)/2 + (Σa*(-2*a + b + c)**2)/2)/(2*(a + b)*(a + c)*(b + c)))


Solution(problem = Σa/(b + c) - 3/2, solution = ((Σa*(b - c)**2)/2 + (Σa*(-2*a + b + c)**2)/2)/(2*(a + b)*(a + c)*(b + c)))

In [73]:
sol = sum_of_squares(CyclicSum(a/(b+c), (a,b,c)) - Rational(3,2), [a,b,c])
print('Type of sol.solution =', type(sol.solution))

display(sol.solution) # the original solution is equal to this
display(sol.solution.doit()) # expand the cyclic sums

# Verify the result is correct (of course it is unless there is a bug)
diff = sol.solution - (a/(b+c)+b/(c+a)+c/(a+b) - Rational(3,2))
print('Is equal =', diff.simplify() == 0)

Type of sol.solution = <class 'sympy.core.mul.Mul'>


((Σa*(b - c)**2)/2 + (Σa*(-2*a + b + c)**2)/2)/(2*(a + b)*(a + c)*(b + c))

(a*(b - c)**2/2 + a*(-2*a + b + c)**2/2 + b*(-a + c)**2/2 + b*(a - 2*b + c)**2/2 + c*(a - b)**2/2 + c*(a + b - 2*c)**2/2)/(2*(a + b)*(a + c)*(b + c))

Is equal = True


In [74]:
# Another display format: not to cancel the denominator:
sol = sum_of_squares(CyclicSum(a/(b+c), (a,b,c)) - Rational(3,2), [a,b,c])
sol.as_eq(cancel=False) # this is a sympy Equality object

Eq(Σa/(b + c) - 3/2, (Σa*(b - c)**2 + Σa*(-2*a + b + c)**2)/(4*(a + b)*(a + c)*(b + c)))


#### Example: Schur's inequality

Goal: Prove that $x^4(x-y)(x-z)+y^4(y-z)(y-x)+z^4(z-x)(z-y)\geq 0$ for all real numbers $x,y,z$.

In [57]:
# Proving Schur's inequality of degree 6 for real numbers x,y,z. There are no inequality or equality constraints.

sum_of_squares(x**4*(x-y)*(x-z) + y**4*(y-z)*(y-x) + z**4*(z-x)*(z-y))

# sum_of_squares(x**4*(x-y)*(x-z) + y**4*(y-z)*(y-x) + z**4*(z-x)*(z-y)).doit() # Try using .doit() to expand cyclic sums

Solution(problem = x**4*(x - y)*(x - z) + y**4*(-x + y)*(y - z) + z**4*(-x + z)*(-y + z), solution = ((4*(Σx)**2/3 + Σ(x - y)**2)*(∏(x - y)**2) + (Σ(x - y)**2*(-3*x**3 + x**2*y + 2*x**2*z + x*y**2 - 4*x*y*z + 3*x*z**2 - 3*y**3 + 2*y**2*z + 3*y*z**2 - 2*z**3)**2)/9)/(Σ(x - y)**2))

In [68]:
# get the tex code of the solution using `.to_string()`
sol = sum_of_squares(x**4*(x-y)*(x-z) + y**4*(y-z)*(y-x) + z**4*(z-x)*(z-y))
string = sol.to_string()
print(string)

print('='*80+'\n'+'='*80)
string2 = sol.doit().to_string() # the string after expanding cyclic sums
print(string2)

\left(x^{4} \left(x - y\right) \left(x - z\right) + y^{4} \left(- x + y\right) \left(y - z\right) + z^{4} \left(- x + z\right) \left(- y + z\right)\right) \sum_{\mathrm{cyc}} \left(x - y\right)^{2} = \frac{\left(4 \left(\sum_{\mathrm{cyc}} x\right)^{2} + 3 \sum_{\mathrm{cyc}} \left(x - y\right)^{2}\right) \prod_{\mathrm{cyc}} \left(x - y\right)^{2}}{3} + \frac{\sum_{\mathrm{cyc}} \left(x - y\right)^{2} \left(- 3 x^{3} + x^{2} y + 2 x^{2} z + x y^{2} - 4 x y z + 3 x z^{2} - 3 y^{3} + 2 y^{2} z + 3 y z^{2} - 2 z^{3}\right)^{2}}{9}
\left(x^{4} \left(x - y\right) \left(x - z\right) + y^{4} \left(- x + y\right) \left(y - z\right) + z^{4} \left(- x + z\right) \left(- y + z\right)\right) \left(\left(- x + z\right)^{2} + \left(x - y\right)^{2} + \left(y - z\right)^{2}\right) = \frac{\left(- x + z\right)^{2} \left(x - y\right)^{2} \left(y - z\right)^{2} \left(3 \left(- x + z\right)^{2} + 3 \left(x - y\right)^{2} + 3 \left(y - z\right)^{2} + 4 \left(x + y + z\right)^{2}\right)}{3} + \frac{\left(

#### Example: 2000 IMO P2

Goal: (2000 IMO P2) Given $a,b,c\geq 0$, $abc=1$, prove that $(a-1+\frac 1b)(b-1+\frac 1c)(c-1+\frac 1a)\leq 1$.

In [8]:
sum_of_squares(1 - (a-1+1/b)*(b-1+1/c)*(c-1+1/a), [a,b,c], [a*b*c-1])

Solution(problem = -(a - 1 + 1/b)*(b - 1 + 1/c)*(c - 1 + 1/a) + 1, solution = ((a*b*c - 1)*(Σ(-3*a**2*b*c**2 + 3*a**2*c**2 - 30*a*b**3*c + 27*a*b**2/2 - 7*a*b*c + 15*a/2 + 16)) + 3*(Σ(a - 1)**2*(3*b**2*c*(b - 1)**2 + b*(-b*c + 1)**2 + 17*b + 9*c*(b - 1)**2*(b - c)**2))/4 + 3*(Σ(a - 1)**2*(6*a*b**2*c + 28*b**4*c + 31*b**3 + 16*b**2 + 16*b*c*(b - 1)**2))/4 + 3*(Σa*(-a + b*c)**2*(6*b**2 + b))/2 + 21*(Σa*b**2*c**2*(b - 1)**2))/(a*b*c*(Σ(17*a*b + 74*b + 16*(1 - b)**2 + 7*(a - b)**2))))

### Properties and Methods of `Solution` class objects:

* `.solution` returns the SymPy expression, which is equal to the input expression
* `.time` returns the solving time
* `.as_eq()` returns a SymPy Equality object, which has `.lhs` and `.rhs` properties.
* `.to_string()` returns the string
* `.doit()` returns a new Solution object with cyclic expressions expanded
* `.xreplace(...)` returns a new Solution by applying `xrepalce` on the solution
* (More to be added)

## Using Alias

Use a python dictionary rather a list to provide the "alias" of a constraint. This helps track the constraints.
It works for both inequality and equality constraints.

In [31]:
# Proving 5 - 3x - 4y >= 0 given no inequality constraint and one equality constraint x^2+y^2=1:
display(sum_of_squares(5 - 3*x - 4*y, [], [x**2+y**2-1]))

# We can also let x^2+y^2-1 = a = 0 by using a dictionary:
display(sum_of_squares(5 - 3*x - 4*y, [], {x**2+y**2-1: a}))

# Let it be a symbol called "(x^{2}+y^{2}-1)", this avoids expanding the terms:
display(sum_of_squares(5 - 3*x - 4*y, [], {x**2+y**2-1: sp.Symbol('(x^{2}+y^{2}-1)')}))

Solution(problem = -3*x - 4*y + 5, solution = -5*x**2/2 - 5*y**2/2 + 9*(5*x/3 - 1)**2/10 + 8*(5*y/4 - 1)**2/5 + 5/2)

Solution(problem = -3*x - 4*y + 5, solution = -5*a/2 + 9*(5*x/3 - 1)**2/10 + 8*(5*y/4 - 1)**2/5)

Solution(problem = -3*x - 4*y + 5, solution = -5*(x^{2}+y^{2}-1)/2 + 9*(5*x/3 - 1)**2/10 + 8*(5*y/4 - 1)**2/5)

In [58]:
# Proving CyclicSum(a**2*(a+b)*(b-c), (a,b,c)) >= 0 given b+c-a>=0, c+a-b>=0, a+b-c >= 0
# We also let F(a), F(b), F(c) = b+c-a, c+a-b, a+b-c by using a dictionary:
F = sp.Function('F')
sum_of_squares(CyclicSum(a**2*(a+b)*(b-c), (a,b,c)), {b+c-a:F(a), c+a-b:F(b), a+b-c:F(c)})

Solution(problem = Σa**2*(a + b)*(b - c), solution = (Σ((-b + c)**2*F(a)*F(c) + 2*(a**2 - 2*a*c + b*c)**2)*F(b))/(2*(ΣF(a))))

## More Advanced Features

More advanced features:

* Squareroots and other algebraic operators (sympy.Abs/Max/Min) are supported.

* Sines, cosines and other trignometric functions are supported if it is algebraic after a change of the variables.

These features are still under active development and are very likely to fail for difficult problems. It is recommended to simplify the problem manually if you can.

In [80]:
# Proving x^3+y^3+z^3-(x^2y+y^2z+z^2x) + (sqrt(13+16sqrt(2))-1)/2*(x-y)(y-z)(z-x) >= 0 for x,y,z >= 0
problem = CyclicSum(x**3,(x,y,z))-CyclicSum(x**2*y,(x,y,z)) + (sqrt(13+16*sqrt(2))-1)/2*(x-y)*(y-z)*(z-x)
sum_of_squares(problem, [x,y,z]) # hint: use .doit() to expand cyclic sums

Solution(problem = (-1/2 + sqrt(13 + 16*sqrt(2))/2)*(-x + z)*(x - y)*(y - z) + Σx**3 - Σx*z**2, solution = (2*(∏x)*(Σ(x - y)**2) + (Σx*(14*y**2 + y*(-x + z)*(-3*sqrt(13 + 16*sqrt(2)) + 7 + sqrt(2)*sqrt(13 + 16*sqrt(2)) + 7*sqrt(2)) - 14*z**2 + z*(x - y)*(-sqrt(2)*sqrt(13 + 16*sqrt(2)) + 7 + 7*sqrt(2) + 3*sqrt(13 + 16*sqrt(2))))**2)/98)/(2*(Σx*y)))

In [98]:
# Proving a*sqrt(4*a**2+5*b*c) + b*sqrt(4*b**2+5*c*a) + c*sqrt(4*c**2+5*a*b) >= (a+b+c)^2 for a,b,c >= 0
problem = a*sqrt(4*a**2+5*b*c) + b*sqrt(4*b**2+5*c*a) + c*sqrt(4*c**2+5*a*b) - (a+b+c)**2
sol = sum_of_squares(problem, [a,b,c])
print('Success =', sol is not None)
display(sol)

# Verify sol.solution == problem when (a,b,c) = (1,4,7) numerically (20 digits):
# (expanding might take too much time)
print('Problem.subs({a:1,b:4,c:7}) =', problem.subs({a:1,b:4,c:7}).simplify().n(20))
print('Solution.subs({a:1,b:4,c:7}) =', sol.solution.subs({a:1,b:4,c:7}).simplify().n(20))

Success = True


Solution(problem = a*sqrt(4*a**2 + 5*b*c) + b*sqrt(5*a*c + 4*b**2) + c*sqrt(5*a*b + 4*c**2) - (a + b + c)**2, solution = ((Σa*(-sqrt(4*a**2 + 5*b*c) + sqrt(5*a*c + 4*b**2))**2*(6983095*b*c*sqrt(5*a*c + 4*b**2) + 15320728*b*(-a + b)**2 + 3007056*(-a + c)**2*sqrt(5*a*c + 4*b**2)))/2 + (Σa*(sqrt(4*a**2 + 5*b*c) - sqrt(5*a*c + 4*b**2))**2*(16687876*b**2*c + 4291307*b**2*sqrt(4*a**2 + 5*b*c) + 1903200*b*sqrt(5*a*b + 4*c**2)*sqrt(5*a*c + 4*b**2) + 133224*sqrt(4*a**2 + 5*b*c)*(-sqrt(4*a**2 + 5*b*c) + sqrt(5*a*b + 4*c**2))**2))/2 + 49914528*(Σa**2*b**2*(a - b)**2) + 12478632*(Σa*b*(a - b)**2*(c - sqrt(5*a*c + 4*b**2))**2) + (Σa*b*(-3*c + sqrt(4*a**2 + 5*b*c))**2*(16144582*c**2 + 19213793*c*sqrt(4*a**2 + 5*b*c)))/2 + 38*(Σb*c**2*(b - c)**2*(79300*sqrt(4*a**2 + 5*b*c) + 100023*sqrt(5*a*c + 4*b**2))) + 4632698*(Σa*b*sqrt(4*a**2 + 5*b*c)*sqrt(5*a*c + 4*b**2)*(sqrt(4*a**2 + 5*b*c) - sqrt(5*a*c + 4*b**2))**2))/(Σ(1421056*a**4 + 24360896*a**3*b + 355264*a**3*sqrt(4*a**2 + 5*b*c) + 57425824*a**2*b**2 

Problem.subs({a:1,b:4,c:7}) = 10.678066681158278314
Solution.subs({a:1,b:4,c:7}) = 10.678066681158278314


In [54]:
# Proving sin(x)+sin(2x)+sin(3x) <= 5/2 given a real number x:
problem = Rational(5,2) - sp.sin(x) - sp.sin(2*x) - sp.sin(3*x)
sol = sum_of_squares(problem)
print('Success =', sol is not None)
display(sol.as_eq(cancel=False))

# Verify sol.solution == problem when x = 2 numerically (20 digits):
# (simplification might take too much time)
print('Problem.subs({x:2}) =', problem.subs({x:2}).simplify().n(20))
print('Solution.subs({x:2}) =', sol.solution.subs({x:2}).simplify().n(20))

Success = True


Eq(-sin(x) - sin(2*x) - sin(3*x) + 5/2, ((-sin(x) + cos(x))**4 + 4*(-sin(x) + cos(x)**2)**2)/(2*(sin(x/2)**2 + cos(x/2)**2)))

Problem.subs({x:2}) = 2.6269205666811724288
Solution.subs({x:2}) = 2.6269205666811724288


In [76]:
# Proving |x|+|y|>=|x+y|
sum_of_squares(sp.Abs(x)+sp.Abs(y)-sp.Abs(x+y)).doit().as_eq(cancel=False)

Eq(Abs(x) + Abs(y) - Abs(x + y), (x*y - Abs(x)*Abs(y))**2/((Abs(x) + Abs(y) + Abs(x + y))*Abs(x)*Abs(y)))